In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data_with_pert import ModelNet40_pert,download,load_data
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from matplotlib import pyplot as plt
import pickle
import warnings
import math
import time
from datetime import datetime
from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from torch import jit
%matplotlib inline
import os
from torch.utils.data import Dataset, TensorDataset
import importlib

In [2]:
train_losses = []
test_results_exp = []
test_accs = []

In [3]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lrt = lr * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lrt

In [56]:
def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
    return torch.index_select(a, dim, order_index)

In [4]:
device = torch.device("cuda:0")

In [19]:
M=10
data = ModelNet40_pert(num_points=2048, partition='train', gaussian_noise=False,
                       unseen=False, factor=4,n_pert=M)
data_test = ModelNet40_pert(num_points=2048, partition='test', gaussian_noise=False,
                       unseen=False, factor=4,n_pert=M)
train_loader = DataLoader(data, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(data_test, batch_size=128, shuffle=True, drop_last=True)
rl_loader = DataLoader(data_test, batch_size=1, shuffle=True, drop_last=True)

/home/gwu/ConvexICP/data_with_pert.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [6]:
def train_eval_icp(model,device, train_loader,test_loader, optimizer,criterion, epoch, eval_mode='on'):
    global best
    model.train()
    final_cal = nn.Sigmoid()
    train_error_logs = []
    t1 = datetime.now()
    test_loss_log = []
    test_acc_log = []
    train_corrects = 0
    
    for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
        src = src.to(device)
        target = target.to(device)
        rotation_ab = rotation_ab.to(device)
        translation_ab = translation_ab.to(device)
        rotation_ba = rotation_ba.to(device)
        translation_ba = translation_ba.to(device)
        optimizer.zero_grad()
        output_train = model(src,target,rotation_ab,translation_ab)
        loss = criterion(output_train, target)
        loss.backward()
        optimizer.step()
        #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
        train_error_logs.append(loss.item())
        
        if (batch_idx+0) % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
                epoch, (batch_idx+0)* len(target), len(train_loader.dataset),
                100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))

        
    print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
                epoch, train_corrects, len(train_loader.dataset),
                100. * train_corrects / len(train_loader.dataset)))
    
    model.eval()
    test_loss = 0
    correct_test = 0
    with torch.no_grad():
        for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba in test_loader:
            src = src.to(device)
            target = target.to(device)
            rotation_ab = rotation_ab.to(device)
            translation_ab = translation_ab.to(device)
            rotation_ba = rotation_ba.to(device)
            translation_ba = translation_ba.to(device)
            output_test = model(src,target,rotation_ab,translation_ab)
            test_loss += criterion(output_test, target).item()*len(src)  # sum up batch loss
    test_loss /= len(test_loader.dataset)
    test_loss_log.append(test_loss)
    print('Test set: Average loss: {:.8f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct_test, len(test_loader.dataset),
            100. * correct_test / len(test_loader.dataset)))
    return train_error_logs,test_loss_log,test_acc_log

In [7]:
def cal_rt_loss(ra,ta,rb,tb):
    r_loss = F.mse_loss(ra,rb).item()
    t_loss = F.mse_loss(ta,tb).item()
    return r_loss, t_loss

In [8]:
class PointNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv5 = nn.Conv1d(128, emb_dims, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(emb_dims)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        return x

In [77]:
class CPNet(nn.Module):
    def __init__(self, emb_dims=512,split = 10):
        super(CPNet, self).__init__()
        self.split = split
        self.alpha = alpha
        self.beta = beta
        self.eps = eps
        self.eta = eta
        self.base_net = PointNet()
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2, R, T, R_pert, T_pert):
        
        trans = torch.matmul(R, x1) + T.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)
        
        pert_out = torch.matmul(R_pert, x1) + T_pert.permute(1,0,2).unsqueeze(3)
        pert_emb1 = self.base_net(pert_out)
        pert_emb1 = pert_emb1.reshape(-1, pter.shape[-2],pter.shape[-1])
        pert_emb2 = tile(emb2,0,self.split)
        pert_dist = self.loss(pert_emb1,pert_emb2)/self.split
        pert_dist = pert_dist.reshape(self.split, emb2.shape[0],emb2.shape[1],emb2.shape[2]).sum(0)
        

        return dist,pert_dist

In [10]:
class CPEvalNet(nn.Module):
    def __init__(self, base_net, emb_dims=512):
        super(CPEvalNet, self).__init__()
        self.base_net = PointNet()
        self.base_net.load_state_dict(base_net.state_dict())
        for param in self.base_net.parameters():
            param.require_grad = False
        self.rotation = nn.Parameter(torch.Tensor(1,3,3))
        self.translation = nn.Parameter(torch.Tensor(1,3))
        nn.init.kaiming_uniform_(self.rotation, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.translation, a=math.sqrt(5))
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2):
        
        trans = torch.matmul(self.rotation, x1) + self.translation.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [89]:
def combine_loss(dist, pert_dist, target, alpha = 1, beta = 1, eps = 0.005, eta = 0.005):
    loss = torch.mean(output)
    
    diff = dist-pert_dist
    dummy = torch.zeros_like(diff)
    pert_loss = F.hinge_embedding_loss(diff,dummy,margin = eps)
    
    return loss+alpha*pert_loss

In [90]:
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [91]:
model = CPNet().to(device)

NameError: name 'alpha' is not defined

In [8]:
np.random.normal(0,0.05,[1,3,3]).shape

(1, 3, 3)

In [6]:
for batch_idx, (src, target, rotation_ab, translation_ab, _,_,_,_, rotation_ab_perts,translation_ab_perts) in enumerate(train_loader):
    break

In [8]:
rotation_ab_perts.shape

torch.Size([128, 10, 3, 3])

In [10]:
rotation_ab.shape

torch.Size([128, 3, 3])

In [29]:
torch.matmul(rotation_ab,src)+translation_ab.unsqueeze(2)

tensor([[[-0.4680, -0.1385, -0.1165,  ..., -0.1123, -0.5325, -0.3342],
         [ 0.0275,  0.3475, -0.3808,  ..., -0.2217, -0.3931, -0.3121],
         [ 0.6172,  0.6456,  0.1607,  ..., -0.0791,  0.1639,  0.0266]],

        [[-0.6744,  0.2956, -0.3846,  ..., -0.5703, -0.0270, -0.4596],
         [ 0.2720,  0.5078,  0.3744,  ...,  0.4553,  0.3439,  0.4805],
         [-0.4871,  0.2221, -0.3216,  ..., -0.2080, -0.0156, -0.6309]],

        [[ 0.8187,  0.7834, -0.0693,  ...,  0.9602,  0.8107,  1.2010],
         [-0.4824, -0.0644,  0.1491,  ..., -0.2072,  0.3462, -0.0713],
         [-0.0613,  0.2017,  0.8395,  ...,  0.0240,  0.3558, -0.0794]],

        ...,

        [[-0.4171, -0.2501, -0.3013,  ..., -0.1062, -0.1476, -0.2350],
         [-0.0558, -0.1257, -0.6043,  ..., -0.6842, -0.1994, -0.1624],
         [ 0.3387, -0.3933,  0.0667,  ..., -0.2515, -0.4481,  0.2950]],

        [[-0.2563, -0.0308,  0.0111,  ..., -0.0865,  0.0434, -0.3073],
         [ 0.0891, -0.0132, -0.3094,  ...,  0.2806,  0.

In [21]:
translation_ab_perts.shape

torch.Size([128, 10, 3])

In [16]:
torch.matmul(rotation_ab_perts.permute(1,0,2,3),src).shape

torch.Size([10, 128, 3, 2048])

In [60]:
pter = torch.matmul(rotation_ab_perts.permute(1,0,2,3),src)+translation_ab_perts.permute(1,0,2).unsqueeze(3)

In [27]:
torch.matmul(rotation_ab_perts[0][0],src[0])+translation_ab_perts[0][0].unsqueeze(1)

tensor([[-0.4895, -0.1660, -0.0937,  ..., -0.0835, -0.5182, -0.3119],
        [ 0.2063,  0.5054, -0.1978,  ..., -0.0494, -0.2008, -0.1296],
        [ 0.6109,  0.6597,  0.1562,  ..., -0.0744,  0.1471,  0.0203]])

In [24]:
translation_ab_perts.unsqueeze(3).shape

torch.Size([128, 10, 3, 1])

In [30]:
pl = torch.matmul(rotation_ab,src)+translation_ab.unsqueeze(2)

In [64]:
pl.shape

torch.Size([128, 3, 2048])

In [48]:
po = pl.repeat(10,1,1)

In [65]:
tile(pl,0,10).shape

torch.Size([1280, 3, 2048])

In [59]:
po.shape

torch.Size([1280, 3, 2048])

In [62]:
pter.shape

torch.Size([10, 128, 3, 2048])

In [71]:
ps = pter.reshape(-1,pter.shape[-2],pter.shape[-1])

In [76]:
ps.reshape(10,128,3,2048).sum(0).shape

torch.Size([128, 3, 2048])

In [78]:
lo = torch.randn(10)

In [81]:
dummy = torch.zeros(10)

In [79]:
lo

tensor([ 0.5358,  0.5636,  0.4102,  1.4853,  0.0739, -1.5253,  0.0799, -1.2627,
        -0.6321, -0.7333])

In [88]:
F.hinge_embedding_loss(lo,dummy,margin = 0,reduction="none")

tensor([0.5358, 0.5636, 0.4102, 1.4853, 0.0739, 0.0000, 0.0799, 0.0000, 0.0000,
        0.0000])

In [84]:
torch.nn.HingeEmbeddingLoss?